Get list of cases in a specified time period by scraping the data table instead of the CSV file

In [5]:
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options

from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
import time

In [6]:
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))
driver.implicitly_wait(3) # seconds
wait = WebDriverWait(driver, 10)

In [7]:
driver.get("https://permitsandcases.indy.gov/")
assert "Accela Citizen Access" in driver.title
elem = driver.find_element(By.XPATH, "//*[@title='Planning / Historic Preservation']")
elem.click()
assert driver.find_element(By.XPATH, '//*[@id="ctl00_PlaceHolderMain_lblPermitListTitle"]').text == 'Search for Planning / IHPC Cases'
elem = driver.find_element(By.ID, "ctl00_PlaceHolderMain_generalSearchForm_txtGSStartDate")
for _ in range(11):
    elem.send_keys(Keys.ARROW_LEFT)
elem.send_keys('09/29/2024')

elem = driver.find_element(By.ID, "ctl00_PlaceHolderMain_generalSearchForm_txtGSEndDate")
for _ in range(11):
    elem.send_keys(Keys.ARROW_LEFT)
elem.send_keys('10/02/2024')
elem = driver.find_element(By.ID, "ctl00_PlaceHolderMain_btnNewSearch").click()
elem = wait.until(EC.visibility_of_element_located((By.ID, "ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList")))

In [24]:
import pandas as pd
from io import StringIO
elem = driver.find_element(By.ID, 'ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList').get_attribute('outerHTML')
#print(elem)
case_listings_frame = pd.read_html(StringIO(elem), skiprows=3)
case_listings_frame[0].to_csv('out.csv', index=False)
#print(case_listings_frame)